## Introduction
Literary authors have a wide variety of styles, contexts, and characters they describe in their works. Many of these styles are repeatable, demonstrating identifiable patterns across their bodies of work. These styles can evolve over time, changing as the lives of the authors themselves evolve. 

## Initialize Your Resources

SageMaker needs unique training jobs to run, and we as the users need to be able to see our job! So here we'll provide our name once, and use that to track our resources throughout the lab.

In [9]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket() # Replace with your own bucket name if needed
print(bucket)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-023375022819


arn:aws:iam::023375022819:role/service-role/AmazonSageMaker-ExecutionRole-20181029T121824
sagemaker-us-east-1-023375022819


## Data Preperation

## Data Preprocessing

## Training

## Hosting / Inference

## Stop / Close the Endpoint (Optional)